In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import itertools
import math
from matplotlib.ticker import AutoMinorLocator
from functools import reduce

f_alg = {
    "SPM": "Small Progress Measures",
    "SCC_FB": "Forward-Backward",
    "SCC_COL": "Colouring/Heads-off",
    "synthesis": "Supervisory Controller Synthesis",
    "prefix_sum": "Prefix Sum"
};

f_order = {
    "relaxed": "Relaxed",
    "acqrel": "Acq/Rel",
    "seqcons": "SeqCons"
};

f_schedule = {
    "in-kernel": "In-kernel",
    "on-host": "On-host",
    "graph": "Graph"
};

# AuDaLa data
spm = pd.read_csv('SPM/SPM_results.csv', sep=',')
scc_col = pd.read_csv('SCC/SCC_COL_results.csv', sep=',')
scc_fb = pd.read_csv('SCC/SCC_FB_results.csv', sep=',')
sorting = pd.read_csv('sorting/sorting_results.csv', sep=',')
prefix_sum = pd.read_csv('prefix_sum/prefix_sum_results.csv', sep=',')
synthesis = pd.read_csv('synthesis/synthesis_results.csv', sep=',')

all_data = [spm, scc_col, scc_fb, sorting, prefix_sum, synthesis];
cols = list(map((lambda d : d.columns), all_data));
assert (all([all(cols[i-1] == cols[i]) for i in range(1, len(cols))]));

data = pd.concat(all_data, ignore_index=True);
data = data.drop(data[data["runtime"] == "timeout"].index);
data["runtime"] = data["runtime"].astype(float);
data["problem_size1"] = data["problem_size1"].astype(int);
data["problem_size2"] = data["problem_size2"].astype(int);
data["problem_size3"] = data["problem_size3"].astype(int);
data["weak_non_racing"] = data["weak_non_racing"].astype(bool);

data["problem_size"] = 0;
data.loc[(data["algorithm"] != "SCC_FB"), "problem_size"] = data[["problem_size1", "problem_size2", "problem_size3"]].max(axis=1);
data.loc[(data["algorithm"] == "SCC_FB"), "problem_size"] = data["problem_size2"];


# Sequential data
synthesis_seq = pd.read_csv('synthesis/synthesis_sequential.csv', sep=',')
spm_seq = pd.read_csv('SPM/SPM_sequential.csv', sep=',')
prefix_sum_seq = pd.read_csv('prefix_sum/prefix_sum_sequential.csv', sep=',')

all_data = [synthesis_seq, spm_seq, prefix_sum_seq];
cols = list(map((lambda d : d.columns), all_data));
assert (all([all(cols[i-1] == cols[i]) for i in range(1, len(cols))]));
seq_data = pd.concat(all_data, ignore_index=True);
seq_data["runtime"] = seq_data["runtime"].astype(float);
seq_data["problem_size1"] = seq_data["problem_size1"].astype(int);
seq_data["problem_size2"] = seq_data["problem_size2"].astype(int);
seq_data["problem_size3"] = seq_data["problem_size3"].astype(int);
seq_data["problem_size"] = seq_data[["problem_size1", "problem_size2", "problem_size3"]].max(axis=1);
seq_data


In [ ]:
def memorder_chart(algs, data, with_only_onrp):
    colors = {
        "relaxed": "C0",
        "acqrel": "C2",
        "seqcons": "C1"
    };
    
    threads_color = "C3";
    threads_ls = "--";#(0, (4, 8));
    threads_alpha = 0.5;
    
    linestyles = {
        True: "-",
        False: (0, (1, 3))
    };
    markers = {
        "graph": ("P", 7),
        "in-kernel": ("D", 5),
        "on-host": ("*", 8)
    };
    
    config_cols = [
        "schedule",
        "voting-strat"
    ];
    
    voting = [
        ("in-kernel", "in-kernel-simple"),
        ("on-host", "on-host-simple"),
        ("graph", "graph-simple")
    ];
    order = ["relaxed", "acqrel", "seqcons"];
    
    ONRP = [True] if with_only_onrp else [True, False];
    
    if with_only_onrp:
        algs.remove(("prefix_sum", "Random"));
    
    fig = plt.figure(constrained_layout=True, figsize=(10, 2.5 * len(algs)));
    subfigs = fig.subfigures(nrows=len(algs), ncols=1)
    
    for a, subfig in enumerate(subfigs):
        (alg, p_type) = algs[a];
        rows = data[(data["algorithm"] == alg) & (data["problem_type"] == p_type)];
        name = f_alg[alg];
        sub_name = f" ({p_type})" if alg == "SPM" else "";
        subfig.suptitle(f'{name}{sub_name}:', fontweight="bold");
        axs = subfig.subplots(nrows=1, ncols=3, sharey=True);
        
        
        prepped = pd.concat([
            rows[
                (rows["schedule"] == s) &
                (rows["voting-strat"] == v) &
                (rows["memorder"] == o) &
                (rows["weak_non_racing"] == opt)
            ][["problem_size", "runtime"]]
            .set_index("problem_size")
            .rename(columns={"runtime": f"rt_{s}_{o}_{opt}"})
            for ((s, v), o, opt) in itertools.product(voting, order, ONRP)
        ], axis = 1);
        prepped.sort_index(inplace=True);

        axs[0].set_ylabel("Relative runtime (%)");
        for ((sf, (s, v)), o, opt) in itertools.product(enumerate(voting), order, ONRP):
            Y = (prepped[f"rt_{s}_{o}_{opt}"] / prepped[f"rt_{s}_relaxed_True"]) * 100;
            axs[sf].plot(
                Y.index,
                Y,
                color=colors[o],
                linestyle=linestyles[opt],
                marker="o" if opt else "*"
            );
            
            if a == 0:
                axs[sf].set_title(f_schedule[s]);

            if a == len(algs)-1:
                axs[sf].set_xlabel("Problem size");
            
            max_resident_threads = 0;
            
            if sf >= 1: # graph || on-host
                max_resident_threads = 70656;
            else: # in-kernel
                if alg == "SPM" or alg == "SCC_COL":
                    max_resident_threads = 58880;
                elif alg == "prefix_sum" or alg == "synthesis":
                    max_resident_threads = 70656;
                elif alg == "SCC_FB":
                    max_resident_threads = 52992;
                else:
                    raise Exception("Unexpected alg");
            
        for i in range(3):
            axs[i].axvline(max_resident_threads, color=threads_color, alpha=threads_alpha, linestyle=threads_ls);
            axs[i].set_xlim(10**3, 10**7);
            axs[i].set_xticks([10**(3+n) for n in range(5)]);
            axs[i].set_xticklabels([f"$10^{3+n}$" for n in range(5)]);
            axs[i].grid(True);
            axs[i].yaxis.set_minor_locator(AutoMinorLocator(2))
            axs[i].set_xscale("log");
            axs[i].tick_params(axis='x', which='minor', bottom=False);

    patches = [
        Line2D(
            [0],
            [0],
            color=colors[o],
            linestyle=linestyles[opt],
            label=f"{f_order[o]}{' (ONRP)' if opt else ''}",
            linewidth=1,
            marker="o" if opt else "*"
        ) for (o, opt) in itertools.product(order, ONRP)
    ];
    
    patches.append(Line2D([0], [0], color=threads_color, alpha=threads_alpha, linestyle=threads_ls, label="Max threads"))
    
    fig.legend(handles=patches, bbox_to_anchor=(1.18, 0.97));

    def prune(name):
        return name.replace(" ", "-").replace("/", "-");
    
    plt.savefig(f"memorder-impact.svg", bbox_inches = 'tight');
    plt.show();


In [ ]:
def schedule_chart(alg, p_type, data, seq_data, relative = False, per_psize = False, schedules = None):
    assert(not(relative and per_psize));
    
    rows = data[(data["algorithm"] == alg) & (data["problem_type"] == p_type)];
    if schedules is not None:
        rows = rows[rows["schedule"].isin(schedules)];

    voting = sorted(list(set(rows.groupby(["schedule", "voting-strat"]).groups.keys())));
    prepped = pd.concat([
        rows[
            (rows["schedule"] == s) &
            (rows["voting-strat"] == v) &
            (rows["memorder"] == "relaxed") &
            (rows["weak_non_racing"] == True)
        ][["problem_size", "runtime"]]
        .set_index("problem_size")
        .sort_index()
        .rename(columns={"runtime": f"rt_{s}_{v}"})
        for (s, v) in voting
    ], axis = 1);
    
    colors = {
        "graph": "C0",
        "on-host": "C1",
        "in-kernel": "C2",
    };
    
    marker = {
        "graph-simple": ("*", 8, "C3"),
        "in-kernel-simple": ("*", 8, "C4"),
        "on-host-simple": ("*", 8, "C5"),
        "on-host-alternating": ("D", 5, "C6"),
        "in-kernel-alternating": ("D", 5, "C7"),
        "graph-shared": ("+", 8, "C8"),
        "graph-shared-banks": ("x", 8, "C9"),
        
    };
    
    fig, ax = plt.subplots(figsize=(6, 4));
    
    rel = ' relative' if relative else '';
    per_instance = ' per instance' if per_psize else '';
    title = (f"The{rel} runtime{per_instance} of schedule and voting strategies compared.\n"
            f"({alg}{f': {p_type}' if not p_type.startswith('Random') else ''})"
            );
    plt.suptitle(title, y=1.07);
    if relative:
        ax.set_ylabel(f"Relative runtime{per_instance} (%)");
    else:
        ax.set_ylabel(f"Runtime{per_instance} (ms)");
        ax.set_yscale("log");
    
    ax.grid(True);
    ax.set_xlim(10**3, 10**7);
    ax.set_xscale("log");
    ax.set_xlabel("Problem size");
    
    for (s, v) in voting:
        if per_psize:
            Y = prepped[f"rt_{s}_{v}"] / prepped.index;
        elif relative:
            Y = (prepped[f"rt_{s}_{v}"] / prepped[f"rt_graph_graph-simple"]) * 100;
        else:
            Y = prepped[f"rt_{s}_{v}"];

        (m, ms, mc) = marker[v];
        ax.plot(
            prepped.index,
            Y,
            label=v,
            color=colors[s],
            marker=m,
            markersize=ms,
            linestyle=':',
            markerfacecolor=mc,
            markeredgecolor=mc
        );
    
    s_data = seq_data[(seq_data["alg"] == alg) & (seq_data["problem_type"] == p_type)];
    if len(s_data.index) > 0:
        ax.plot(
            s_data["problem_size"],
            s_data["runtime"],
            label="Sequential",
            color="C10"
        );
    
    plt.legend(bbox_to_anchor=(1.04, 1), loc="upper left");
    plt.show();

In [ ]:
algs = [
    ("synthesis", "Random"),
    ("prefix_sum", "Random"),
    ("SPM", "Invariantly Inevitably Eat"),
    ("SPM", "Invariantly Plato Starves"),
    ("SCC_FB", "Random Graph (p=1.3/n)"),
    ("SCC_COL", "Random Graph (p=1.3/n)")
    #("sorting", "Random")
]
memorder_chart(algs, data, with_only_onrp=False);


In [ ]:
for (alg, p_type) in algs:
    schedule_chart(alg, p_type, data, seq_data, relative=False, per_psize=False);
